In [6]:
import torch
import torchvision
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import time

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)

In [8]:
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

In [9]:
b_size = 128
train_loader = torch.utils.data.DataLoader(train_data, batch_size=b_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=b_size, shuffle=True)

In [10]:
images = []
labels = []
for i in range(100):
    image, label = train_data[i]
    if label not in labels:
        labels.append(label)
        images.append(image)
print('Number of classes:', len(labels))
print('Image size:', images[0].shape)

Number of classes: 10
Image size: torch.Size([1, 28, 28])


Checking whether the data is balanced to chose loss function correctly.

In [12]:
targets = train_data.targets
targets2 = test_data.targets

print("Train set: \n", targets.unique(return_counts = True)[1], '\n')
print("Test set: \n", targets2.unique(return_counts = True)[1])

Train set: 
 tensor([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000]) 

Test set: 
 tensor([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])


In [13]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(128, 10)
)

In [14]:
loss = torch.nn.CrossEntropyLoss()
training = optim.Adam(model.parameters(), lr = .01)
num_epochs = 10

In [15]:
def trainer(model, loss, training, num_epochs):
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    for epoch in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train_loader:
            training.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            training.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test_loader:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        train_losses.append(train_loss / train_iters) 
        test_losses.append(test_loss / test_iters)
        train_accs.append(train_acc / train_passed)
        test_accs.append(test_acc / test_passed)
            
        print("Epoch: {}\n Time taken: {:.2f}\n train_loss: {}, train_acc: {} \n test_loss: {}, test_acc: {}\n".format(
            epoch+1, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )
        
    eps = range(1, num_epochs+1)
    plt.plot(eps, train_losses, label='Train Loss')
    plt.plot(eps, test_losses, label='Test Loss')
    plt.plot(eps, train_accs, label='Train acc')
    plt.plot(eps, test_accs, label='Test acc')
    plt.xlabel('Epochs')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    plt.show()

In [18]:
model1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    #torch.nn.Dropout(p=0.4),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    #torch.nn.Dropout(p=0.3),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(128),
    #torch.nn.Dropout(p=0.3),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(64),
    #torch.nn.Dropout(p=0.3),
    torch.nn.Linear(64,10)
    
)

training1 = torch.optim.Adam(model1.parameters(), lr=.01)
num_epochs1 = 15

In [ ]:
trainer(model1, loss, training1, num_epochs1)

Epoch: 1
 Time taken: 13.76
 train_loss: 0.5097837803333298, train_acc: 0.8136333333333333 
 test_loss: 0.49093989820419986, test_acc: 0.8377

Epoch: 2
 Time taken: 14.70
 train_loss: 0.3974763386285127, train_acc: 0.8550333333333333 
 test_loss: 0.5175851402403433, test_acc: 0.8485

Epoch: 3
 Time taken: 14.61
 train_loss: 0.3602225234322965, train_acc: 0.86845 
 test_loss: 0.4737863121907922, test_acc: 0.8587

Epoch: 4
 Time taken: 14.92
 train_loss: 0.33999101375974317, train_acc: 0.8748666666666667 
 test_loss: 1.1486538314743886, test_acc: 0.8508

Epoch: 5
 Time taken: 15.10
 train_loss: 0.3216397623454076, train_acc: 0.8815833333333334 
 test_loss: 0.8468356038196178, test_acc: 0.8629

Epoch: 6
 Time taken: 15.72
 train_loss: 0.30816897594217046, train_acc: 0.8867166666666667 
 test_loss: 1.561607850503318, test_acc: 0.8711

Epoch: 7
 Time taken: 15.35
 train_loss: 0.296114058430388, train_acc: 0.8896333333333334 
 test_loss: 0.8868862879049929, test_acc: 0.8637

Epoch: 8
 Time t